In Sections 5.3.2 and 5.3.3, we saw that the cv.glm() function can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just the glm() and predict.glm() functions, and a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the Weekly data set. Recall that in the context of classification problems, the LOOCV error is given in (5.4).

(a) Fit a logistic regression model that predicts Direction using Lag1 and Lag2.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

# 데이터 로드
data = pd.read_csv("C:/Users/bikmi/Desktop/Bigdata_HW/HW4/Weekly.csv")

# 독립 변수와 종속 변수 정의
X = data[["Lag1", "Lag2"]]
y = data["Direction"].apply(
    lambda x: 1 if x == "Up" else 0
)  # 'Up'을 1로, 'Down'을 0으로 변환

# Leave-One-Out Cross-Validation 설정
loo = LeaveOneOut()
errors = []

# LOOCV 수행
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # 로지스틱 회귀 모델 학습
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # 예측 및 오류 계산
    y_pred = model.predict(X_test)
    errors.append(1 if y_pred != y_test.values else 0)

# LOOCV 오류율 계산
loocv_error_rate = sum(errors) / len(errors)
print(f"LOOCV 오류율: {loocv_error_rate}")

LOOCV 오류율: 0.44995408631772266


(b) Fit a logistic regression model that predicts Direction using Lag1 and Lag2 using all but the first observation.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 데이터 로드
data = pd.read_csv("C:/Users/bikmi/Desktop/Bigdata_HW/HW4/Weekly.csv")

# 첫 번째 관측치를 제외한 데이터셋
X_train = data[["Lag1", "Lag2"]].iloc[1:]  # 첫 번째 행 제외
y_train = data["Direction"].iloc[1:].apply(lambda x: 1 if x == "Up" else 0)

# 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

(c) Use the model from (b) to predict the direction of the first observation. You can do this by predicting that the first observation will go up if P(Direction="Up"∣Lag1,Lag2)>0.5. Was this observation correctly classified?

In [ ]:
import pandas as pd

# 첫 번째 관측치의 Lag1, Lag2 값 추출 및 데이터프레임으로 변환
X_test = pd.DataFrame(data[["Lag1", "Lag2"]].iloc[0]).transpose()
y_true = (
    1 if data["Direction"].iloc[0] == "Up" else 0
)  # 실제 값 ("Up"을 1로, "Down"을 0으로 변환)

# 첫 번째 관측치에 대한 예측 확률 계산
y_prob = model.predict_proba(X_test)[0, 1]  # "Up"에 대한 확률
y_pred = (
    1 if y_prob > 0.5 else 0
)  # 확률이 0.5보다 크면 "Up" (1), 그렇지 않으면 "Down" (0)으로 예측

# 결과 확인
print(f"첫 번째 관측치의 실제 값: {'Up' if y_true == 1 else 'Down'}")
print(f"첫 번째 관측치의 예측 값: {'Up' if y_pred == 1 else 'Down'}")
print(f"첫 번째 관측치의 예측 확률 (Up): {y_prob}")
print("첫 번째 관측치가 올바르게 분류되었는지:", y_pred == y_true)

첫 번째 관측치의 실제 값: Down
첫 번째 관측치의 예측 값: Up
첫 번째 관측치의 예측 확률 (Up): 0.5713809215379875
첫 번째 관측치가 올바르게 분류되었는지: False


(d) Write a for loop from 
i=1 to i=n, where n is the number of observations in the data set, that performs each of the following steps:
i. Fit a logistic regression model using all but the ith observation to predict Direction using Lag1 and Lag2.
ii. Compute the posterior probability of the market moving up for the ith observation.
iii. Use the posterior probability for the ith observation in order to predict whether or not the market moves up.
iv. Determine whether or not an error was made in predicting the direction for the 
ith observation. If an error was made, then indicate this as a 1, and otherwise indicate it as a 0.


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 데이터 로드
data = pd.read_csv("C:/Users/bikmi/Desktop/Bigdata_HW/HW4/Weekly.csv")

# 종속 변수 'Direction'을 이진 형식으로 변환 ('Up'을 1, 'Down'을 0으로 설정)
y = data["Direction"].apply(lambda x: 1 if x == "Up" else 0)

# 오류 리스트 초기화
errors = []

# LOOCV 반복
for i in range(len(data)):
    # i번째 관측치를 제외한 데이터셋 설정 (i번 째 관측치를 테스트 데이터로 사용)

    # i. 로지스틱 회귀 모델을 i번째 관측치를 제외한 데이터셋으로 학습
    X_train = data[["Lag1", "Lag2"]].drop(index=i)
    y_train = y.drop(index=i)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    # ii. i번째 관측치에 대한 시장 상승 확률 계산 (후확률)
    X_test = data[["Lag1", "Lag2"]].iloc[i].values.reshape(1, -2)
    y_prob = model.predict_proba(X_test)[0, 1]  # "Up"에 대한 확률

    # iii. 후확률을 사용하여 시장 상승 여부 예측
    y_pred = 1 if y_prob > 0.5 else 0  # 0.5보다 크면 "Up" (1), 그렇지 않으면 "Down" (0)
    y_true = y.iloc[i]  # 실제 값

    # iv. 예측이 올바른지 여부 확인 후 오류 기록
    errors.append(1 if y_pred != y_true else 0)

# 각 관측치에 대한 오류 여부 출력
print("각 관측치에 대한 오류 여부 (1: 오류, 0: 정확):", errors)

# LOOCV 오류율 계산
loocv_error_rate = sum(errors) / len(errors)
print(f"LOOCV 오류율: {loocv_error_rate}")

c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\L

각 관측치에 대한 오류 여부 (1: 오류, 0: 정확): [1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0

c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-dTzZVxnG-py3.11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\bikmi\AppData\L

(e) Take the average of the n numbers obtained in (d)iv in order to obtain the LOOCV estimate for the test error. Comment on the results.

In [12]:
# LOOCV 오류율 계산
loocv_error_rate = sum(errors) / len(errors)
print(f"LOOCV 오류율: {loocv_error_rate}")

LOOCV 오류율: 0.44995408631772266
